# Seismic Detection

## Part 1: Import dependencies

In [31]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from filter import filter

## Part 2: Select the dataset

If you want to use the directories listed below, make sure you unzip the [Space Apps 2024 Seismic Detection Data Packet](https://wufs.wustl.edu/SpaceApps/data/space_apps_2024_seismic_detection.zip) in the root folder of the project.

In [32]:
# Available directories
lunar_S12B = './space_apps_2024_seismic_detection/data/lunar/test/data/S12_GradeB/'
lunar_S15A = './space_apps_2024_seismic_detection/data/lunar/test/data/S15_GradeA/'
lunar_S15B = './space_apps_2024_seismic_detection/data/lunar/test/data/S15_GradeB/'
lunar_S16A = './space_apps_2024_seismic_detection/data/lunar/test/data/S16_GradeA/'
lunar_S16B = './space_apps_2024_seismic_detection/data/lunar/test/data/S16_GradeB/'
mars = './space_apps_2024_seismic_detection/data/mars/test/data/'

# Select data directory or specify path to a different one
data_directory = mars

# List to store detected seismic events
export = []


## Part 3: Detect and plot seismic events

In [ ]:
# Iterate though files in directory
for filename in os.listdir(data_directory):
    # Look thorugh mSEED files
    if filename.endswith(".mseed"):
        mseed_file = f'{data_directory}{filename}'
        st = read(mseed_file)

        # Process the trace
        tr = st.traces[0].copy()
        tr_times = tr.times()
        tr_data = filter(st)

        first_nonzero_index = np.nonzero(tr_data)[0][0]

        # Start time of trace
        starttime = tr.stats.starttime.datetime

        # Initialize figure
        fig,ax = plt.subplots(1,1,figsize=(10,3))

        # Plot trace
        ax.plot(tr_times,tr_data)

        predicted_time = tr_times[first_nonzero_index]

        # Mark detection
        ax.axvline(x = predicted_time, color='green',label='Detect. Arrival')
        ax.legend(loc='upper left')

        # Add data to the export list
        export_row = {'filename': filename, 'time_abs(%Y-%m-%dT%H:%M:%S.%f)': starttime + timedelta(seconds = predicted_time), 'time_rel(sec)': tr_times[first_nonzero_index]}
        export.append(export_row)

        # Make the plot pretty
        ax.set_xlim([min(tr_times),max(tr_times)])
        ax.set_ylabel('Velocity (m/s)')
        ax.set_xlabel('Time (s)')
        ax.set_title(f'{filename}', fontweight='bold')
        plt.show()

## Part 4: Export detected seismic events

In [34]:
export_df = pd.DataFrame(export)
export_df.to_csv('output.csv', index=False)